In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image 
from sklearn.model_selection import train_test_split

In [ ]:
withmask = os.listdir('/kaggle/input/face-mask-dataset/data/with_mask')
print (withmask[:7])

In [ ]:
withoutmask = os.listdir('/kaggle/input/face-mask-dataset/data/without_mask')
print(withoutmask[:7])

In [ ]:
print('number of withmask is ',len(withmask))
print('number of withoutmask is ',len(withoutmask))

In [ ]:
withmask_labels = [1]*len(withmask)
withoutmask_labels = [0]*len(withoutmask)

In [ ]:
print(withmask_labels[0:5])

print(withoutmask_labels[0:5])

In [ ]:
labels = withmask_labels +withoutmask_labels
print(len(labels))

In [ ]:
img = mpimg.imread('/kaggle/input/face-mask-dataset/data/with_mask/with_mask_1003.jpg')
mgplot = plt.imshow(img)
plt.show()

In [ ]:
img2 = mpimg.imread('/kaggle/input/face-mask-dataset/data/without_mask/without_mask_1005.jpg')
mgplot = plt.imshow(img2)
plt.show()

In [ ]:
data = []
labels = []

withoutmask_path= '/kaggle/input/face-mask-dataset/data/without_mask'
withmask_path = '/kaggle/input/face-mask-dataset/data/with_mask'
# Without mask
for img_file in os.listdir(withoutmask_path):
    image = Image.open(os.path.join(withoutmask_path, img_file)).resize((128,128)).convert('RGB')
    data.append(np.array(image))
    labels.append(0)  # 0 = no mask

# With mask
for img_file in os.listdir(withmask_path):
    image = Image.open(os.path.join(withmask_path, img_file)).resize((128,128)).convert('RGB')
    data.append(np.array(image))
    labels.append(1)  # 1 = mask

print("Total images:", len(data))


In [ ]:
data[0]

In [ ]:
X = np.array(data)
Y = np.array(labels)

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [ ]:
X_train_sacled = X_train/255
X_test_sacled = X_test/255

In [ ]:
X_train[0]

In [ ]:
X_train_sacled[0]

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
num_of_classes = 2

model = keras.Sequential()

model.add(keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(128,128,3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))


model.add(keras.layers.Dense(num_of_classes, activation='sigmoid'))

In [ ]:
history = model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [ ]:
history = model.fit(X_train_sacled, Y_train, validation_split=0.1, epochs=5)

In [ ]:
loss, accuracy = model.evaluate(X_test_sacled, Y_test)
print('Test Accuracy =', accuracy)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# ---- Accuracy Plot ----
plt.figure(figsize=(8, 6))
plt.plot(epochs, acc, 'bo-', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b-', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# ---- Loss Plot ----
plt.figure(figsize=(8, 6))
plt.plot(epochs, loss, 'ro-', label='Training Loss')
plt.plot(epochs, val_loss, 'r-', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
model.save('mask_classifier.h5')

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("mask_classifier.h5")


In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# ✅ Load your trained model
model = load_model('/kaggle/working/mask_classifier.h5')  # update path if needed

# ✅ Base dataset path
base_path = '/kaggle/input/face-mask-dataset/data'
img_size = (128, 128)

# ✅ Collect all image paths from both subfolders
img_files = []
for subdir, _, files in os.walk(base_path):
    for f in files:
        if f.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_files.append(os.path.join(subdir, f))

print(f"✅ Found {len(img_files)} images to test.")


for img_path in img_files[4990:5000]:
    # Load and preprocess
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    pred = model.predict(img_array, verbose=0)
    is_mask = pred[0][0] < 0.5
    label = "😷 WITH MASK" if is_mask else "😶 WITHOUT MASK"
    confidence = pred[0][0] * 100 if is_mask else (1 - pred[0][0]) * 100

    # Display
    plt.imshow(img)
    plt.title(f"{label}")
    plt.axis('off')
    plt.show()
